# Table Building

## Setup

In [1]:
import numpy as np
import pandas as pd
import re
import json
import nltk
from glob import glob
from numpy.linalg import norm

## Useful functions

In [2]:
def get_BOW(corpus:pd.DataFrame, level:str):
    '''
    Function to get bag of words from a corpus

    Corpus here is loosely defined, as this function will work provided the `level` parameter is in the multi-index of `corpus`

    PARAMETERS:

    `corpus` - pandas DataFrame of body of work. It must be multi-indexed by an OHCO

    `level` - string of OHCO to group by for bags

    OUTPUTS:

    pandas DataFrame of bag of words grouiped by `level`

    EXAMPLE:

    `BOW = get_BOW(CORPUS, 'chap_id')`
    '''
    # Get multi-index from `corpus` df
    idx = list(corpus.index.names)

    # Check to see that `level` exists in `corpus` OHCO
    # Raise error if not
    if (level not in idx):
        raise KeyError (f'{level} not found in corpus OHCO')

    # Split-apply-combine to generate BOW grouped by `level`
    return corpus.groupby(idx[:idx.index(level)+1]+['term_str'])\
        .term_str\
        .count()\
        .to_frame('n')

In [3]:
def get_TFIDF(bow:pd.DataFrame, tf_type:str):
    '''
    Function to compute TFIDF and DFIDF for a given bag of words DataFrame

    PARAMETERS:

    `bow` - pandas DataFrame representation of bag of words
    
    `tf_type` - string of term frequency type to use. Options are currently:\n
                sum, max, log, raw, double_norm (defaults to k=1), and binary

    OUTPUTS:
    
    tf-idf vectorized DataFrame
    df-idf vector for terms in vocabulary, can be attached to VOCAB table
    DTCM matrix as a DataFrame

    EXAMPLE:
    `TFIDF, dfidf, DTCM = get_TFIDF(BOW, 'max')`
    '''

    DTCM = bow.n.unstack(fill_value=0)

    # Term frequency calculation dictionary
    if tf_type == 'sum':
        TF = DTCM.T / DTCM.T.sum()

    elif tf_type == 'max':
        TF = DTCM.T / DTCM.T.max()
        
    elif tf_type == 'log':
        TF = np.log2(1 + DTCM.T)
        
    elif tf_type == 'raw':
        TF = DTCM.T
        
    elif tf_type == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
        
    elif tf_type == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
        
    TF = TF.T
    
    # Calculate document frequency
    DF = DTCM.astype('bool').sum()
    
    # Calculate number of documents
    N = DTCM.shape[0]

    # idf calculations
    IDF = np.log2(N / DF)
    
    #TFIDF = TF*IDF

    return (TF*IDF), (DF*IDF), DTCM

## Read Data

In [4]:
with open('../data/cleaned_data.json') as f:
    data = json.load(f)
f.close()

## Build `song_lib` table
This might be useful for getting song info later, but ultimately our library will be based on albums as being the "books" with songs as "chapters"

### Initialize dictionary we'll use to make the data frame later

In [5]:
song_lib_dict = dict()

### Loop over json and add songs and metadata to `lib_dict` with an arbitrary song ID

In [6]:
idx = 0
# Loop over artists
for artist in data.keys():
    # Loop over albums
    for album in data[artist]['Albums'].keys():
        # Loop over tracks
        for track in data[artist]['Albums'][album]['tracklist'].keys():
            song_lib_dict[idx] = {
                'title':track,
                'json_path':(f'data[{artist}][\'Albums\'][{album}][\'tracklist\'][track]'),
                'artist':artist,
                'album':album,
                'genres':data[artist]['Metadata']['genres'],
                'release_date':data[artist]['Albums'][album]['release_date'],
                'label':data[artist]['Albums'][album]['label'],
                'track_number':data[artist]['Albums'][album]['tracklist'][track]['track_number'],
                'duration_ms':data[artist]['Albums'][album]['tracklist'][track]['duration_ms'],
                'track_number':data[artist]['Albums'][album]['tracklist'][track]['track_number'],
                'audio_information':data[artist]['Albums'][album]['tracklist'][track]['audio_information'],
                'lyrics':data[artist]['Albums'][album]['tracklist'][track]['lyrics']
            }
            idx += 1

### Convert `lib_dict` to data frame

In [7]:
song_lib = pd.DataFrame.from_dict(song_lib_dict).T
song_lib.index.name = 'song_id'

In [8]:
song_lib.head()

,title,json_path,artist,album,genres,release_date,label,track_number,duration_ms,audio_information,lyrics
song_id,,,,,,,,,,,
0,Pay Your Way In Pain,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,1,183880,"{'danceability': 0.677, 'energy': 0.826, 'key'...",\n(Ow)\nOh-oh-oh\n\nYou got to pay your way in...
1,Down And Out Downtown,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,2,222240,"{'danceability': 0.61, 'energy': 0.502, 'key':...",\nLast night's heels\nOn the mornin' train\nIt...
2,Daddy's Home,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,3,199613,"{'danceability': 0.774, 'energy': 0.372, 'key'...",\nI signed autographs in the visitation room\n...
3,Live In The Dream,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,4,389546,"{'danceability': 0.426, 'energy': 0.339, 'key'...",\nHello\nDo you know where you are?\nYou've be...
4,The Melting Of The Sun,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,5,257519,"{'danceability': 0.52, 'energy': 0.463, 'key':...","\nSo sorry, missed the party\nHello, on the da..."


### Expand `audio_information` to individual columns

In [9]:
audio_info = song_lib.audio_information

In [10]:
audio_frame = audio_info.apply(pd.Series)

In [11]:
audio_frame.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
song_id,,,,,,,,,,,,,,,,,,
0,0.677,0.826,9,-6.800,1,0.3680,0.3950,0.00000,0.5710,0.4980,124.821,audio_features,1nLaoWVhNRZ38irY0LZjL6,spotify:track:1nLaoWVhNRZ38irY0LZjL6,https://api.spotify.com/v1/tracks/1nLaoWVhNRZ3...,https://api.spotify.com/v1/audio-analysis/1nLa...,183880,4
1,0.610,0.502,6,-8.409,1,0.0820,0.0578,0.01410,0.1660,0.6730,151.906,audio_features,4htB9ArZCpquXlXnkKjAgk,spotify:track:4htB9ArZCpquXlXnkKjAgk,https://api.spotify.com/v1/tracks/4htB9ArZCpqu...,https://api.spotify.com/v1/audio-analysis/4htB...,222240,4
2,0.774,0.372,11,-9.796,0,0.0956,0.6150,0.00148,0.1060,0.6020,110.036,audio_features,6DeE4f5DFeWycZz8f6UEht,spotify:track:6DeE4f5DFeWycZz8f6UEht,https://api.spotify.com/v1/tracks/6DeE4f5DFeWy...,https://api.spotify.com/v1/audio-analysis/6DeE...,199613,4
3,0.426,0.339,2,-10.262,1,0.0270,0.7510,0.06450,0.0871,0.0689,80.224,audio_features,27JieQ9wlcYMf1SwTczPUl,spotify:track:27JieQ9wlcYMf1SwTczPUl,https://api.spotify.com/v1/tracks/27JieQ9wlcYM...,https://api.spotify.com/v1/audio-analysis/27Ji...,389547,4
4,0.520,0.463,10,-10.429,1,0.0958,0.6090,0.00140,0.4260,0.4690,159.931,audio_features,3WD3w5uSzAGJWrNFnHhi1a,spotify:track:3WD3w5uSzAGJWrNFnHhi1a,https://api.spotify.com/v1/tracks/3WD3w5uSzAGJ...,https://api.spotify.com/v1/audio-analysis/3WD3...,257520,4


#### Drop some redundant or unnecessary columns

In [12]:
audio_frame.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [13]:
audio_frame = audio_frame.drop(['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms'], axis=1)

In [14]:
audio_frame.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
song_id,,,,,,,,,,,,
0,0.677,0.826,9,-6.800,1,0.3680,0.3950,0.00000,0.5710,0.4980,124.821,4
1,0.610,0.502,6,-8.409,1,0.0820,0.0578,0.01410,0.1660,0.6730,151.906,4
2,0.774,0.372,11,-9.796,0,0.0956,0.6150,0.00148,0.1060,0.6020,110.036,4
3,0.426,0.339,2,-10.262,1,0.0270,0.7510,0.06450,0.0871,0.0689,80.224,4
4,0.520,0.463,10,-10.429,1,0.0958,0.6090,0.00140,0.4260,0.4690,159.931,4


#### Add `audio_frame` to `lib`

In [15]:
song_lib = song_lib.join(audio_frame)

In [16]:
song_lib.head()

,title,json_path,artist,album,genres,release_date,label,track_number,duration_ms,audio_information,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
song_id,,,,,,,,,,,,,,,,,,,,,
0,Pay Your Way In Pain,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,1,183880,"{'danceability': 0.677, 'energy': 0.826, 'key'...",...,9,-6.800,1,0.3680,0.3950,0.00000,0.5710,0.4980,124.821,4
1,Down And Out Downtown,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,2,222240,"{'danceability': 0.61, 'energy': 0.502, 'key':...",...,6,-8.409,1,0.0820,0.0578,0.01410,0.1660,0.6730,151.906,4
2,Daddy's Home,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,3,199613,"{'danceability': 0.774, 'energy': 0.372, 'key'...",...,11,-9.796,0,0.0956,0.6150,0.00148,0.1060,0.6020,110.036,4
3,Live In The Dream,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,4,389546,"{'danceability': 0.426, 'energy': 0.339, 'key'...",...,2,-10.262,1,0.0270,0.7510,0.06450,0.0871,0.0689,80.224,4
4,The Melting Of The Sun,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,5,257519,"{'danceability': 0.52, 'energy': 0.463, 'key':...",...,10,-10.429,1,0.0958,0.6090,0.00140,0.4260,0.4690,159.931,4


### Change `dtypes` to reasonable types

In [17]:
song_lib.dtypes

title                 object
json_path             object
artist                object
album                 object
genres                object
release_date          object
label                 object
track_number          object
duration_ms           object
audio_information     object
lyrics                object
danceability         float64
energy               float64
key                    int64
loudness             float64
mode                   int64
speechiness          float64
acousticness         float64
instrumentalness     float64
liveness             float64
valence              float64
tempo                float64
time_signature         int64
dtype: object

In [18]:
song_lib = song_lib.astype({
    'track_number':'int16',
    'duration_ms':'int32'
})

In [19]:
song_lib.dtypes

title                 object
json_path             object
artist                object
album                 object
genres                object
release_date          object
label                 object
track_number           int16
duration_ms            int32
audio_information     object
lyrics                object
danceability         float64
energy               float64
key                    int64
loudness             float64
mode                   int64
speechiness          float64
acousticness         float64
instrumentalness     float64
liveness             float64
valence              float64
tempo                float64
time_signature         int64
dtype: object

## Build `lib` table
This is based on albums as "books" and songs as "chapters"

In [20]:
LIB = song_lib.groupby('album').agg({
    'artist':'first',
    'genres':'first',
    'release_date':'first',
    'label':'first',
    'danceability':'mean',
    'energy':'mean',
    'loudness':'mean',
    'speechiness':'mean',
    'acousticness':'mean',
    'instrumentalness':'mean',
    'liveness':'mean',
    'valence':'mean',
    'tempo':'mean'
})

In [21]:
LIB = LIB.rename(columns={
    'artist':'artist',
    'genres':'genres',
    'release_date':'release_date',
    'label':'label',
    'danceability':'mean_danceability',
    'energy':'mean_energy',
    'loudness':'mean_loudness',
    'speechiness':'mean_speechiness',
    'acousticness':'mean_acousticness',
    'instrumentalness':'mean_instrumentalness',
    'liveness':'mean_liveness',
    'valence':'mean_valence',
    'tempo':'mean_tempo'
})

In [22]:
LIB = LIB.sort_index(ascending=True).reset_index()
LIB.index.name = 'album_id'
LIB.head()

,album,artist,genres,release_date,label,mean_danceability,mean_energy,mean_loudness,mean_speechiness,mean_acousticness,mean_instrumentalness,mean_liveness,mean_valence,mean_tempo
album_id,,,,,,,,,,,,,,
0,45:33,LCD Soundsystem,"[alternative dance, alternative rock, art pop,...",2007-11-12,Parlophone UK,0.625000,0.918000,-5.810000,0.074050,0.177000,0.672500,0.105000,0.456000,104.008500
1,A Moon Shaped Pool,Radiohead,"[alternative rock, art rock, melancholia, oxfo...",2016-05-08,XL Recordings,0.429545,0.406455,-11.596818,0.034264,0.673291,0.564455,0.158773,0.203018,125.244000
2,Actor,St. Vincent,"[art pop, etherpop, indie rock, indietronica, ...",2009-05-05,4AD,0.484091,0.548455,-10.276273,0.039882,0.324571,0.433091,0.214909,0.329773,119.461091
3,Al Mundo Azul,Mr. Twin Sister,[chillwave],2021-11-19,Twin Group,0.722100,0.698500,-6.106500,0.054780,0.275440,0.330606,0.191360,0.716500,125.701400
4,Alligator,The National,"[chamber pop, indie folk, indie rock, indietro...",2005-04-11,Beggars Banquet,0.522154,0.740769,-6.816615,0.035585,0.284209,0.239670,0.125769,0.535831,125.053308


In [23]:
LIB = LIB.rename(columns={'album':'album_title'})
LIB.head()

,album_title,artist,genres,release_date,label,mean_danceability,mean_energy,mean_loudness,mean_speechiness,mean_acousticness,mean_instrumentalness,mean_liveness,mean_valence,mean_tempo
album_id,,,,,,,,,,,,,,
0,45:33,LCD Soundsystem,"[alternative dance, alternative rock, art pop,...",2007-11-12,Parlophone UK,0.625000,0.918000,-5.810000,0.074050,0.177000,0.672500,0.105000,0.456000,104.008500
1,A Moon Shaped Pool,Radiohead,"[alternative rock, art rock, melancholia, oxfo...",2016-05-08,XL Recordings,0.429545,0.406455,-11.596818,0.034264,0.673291,0.564455,0.158773,0.203018,125.244000
2,Actor,St. Vincent,"[art pop, etherpop, indie rock, indietronica, ...",2009-05-05,4AD,0.484091,0.548455,-10.276273,0.039882,0.324571,0.433091,0.214909,0.329773,119.461091
3,Al Mundo Azul,Mr. Twin Sister,[chillwave],2021-11-19,Twin Group,0.722100,0.698500,-6.106500,0.054780,0.275440,0.330606,0.191360,0.716500,125.701400
4,Alligator,The National,"[chamber pop, indie folk, indie rock, indietro...",2005-04-11,Beggars Banquet,0.522154,0.740769,-6.816615,0.035585,0.284209,0.239670,0.125769,0.535831,125.053308


### Create `album_name` column with no spaces to be able to reference files

In [24]:
LIB['album_name'] = LIB['album_title'].apply(lambda x: re.sub(r' ', '_', x))
LIB.head()

,album_title,artist,genres,release_date,label,mean_danceability,mean_energy,mean_loudness,mean_speechiness,mean_acousticness,mean_instrumentalness,mean_liveness,mean_valence,mean_tempo,album_name
album_id,,,,,,,,,,,,,,,
0,45:33,LCD Soundsystem,"[alternative dance, alternative rock, art pop,...",2007-11-12,Parlophone UK,0.625000,0.918000,-5.810000,0.074050,0.177000,0.672500,0.105000,0.456000,104.008500,45:33
1,A Moon Shaped Pool,Radiohead,"[alternative rock, art rock, melancholia, oxfo...",2016-05-08,XL Recordings,0.429545,0.406455,-11.596818,0.034264,0.673291,0.564455,0.158773,0.203018,125.244000,A_Moon_Shaped_Pool
2,Actor,St. Vincent,"[art pop, etherpop, indie rock, indietronica, ...",2009-05-05,4AD,0.484091,0.548455,-10.276273,0.039882,0.324571,0.433091,0.214909,0.329773,119.461091,Actor
3,Al Mundo Azul,Mr. Twin Sister,[chillwave],2021-11-19,Twin Group,0.722100,0.698500,-6.106500,0.054780,0.275440,0.330606,0.191360,0.716500,125.701400,Al_Mundo_Azul
4,Alligator,The National,"[chamber pop, indie folk, indie rock, indietro...",2005-04-11,Beggars Banquet,0.522154,0.740769,-6.816615,0.035585,0.284209,0.239670,0.125769,0.535831,125.053308,Alligator


### Create `source_file_path` column

In [25]:
LIB['source_file_path'] = LIB.album_name.apply(lambda x: f'../data/albums/{x}.txt')
LIB.head()

,album_title,artist,genres,release_date,label,mean_danceability,mean_energy,mean_loudness,mean_speechiness,mean_acousticness,mean_instrumentalness,mean_liveness,mean_valence,mean_tempo,album_name,source_file_path
album_id,,,,,,,,,,,,,,,,
0,45:33,LCD Soundsystem,"[alternative dance, alternative rock, art pop,...",2007-11-12,Parlophone UK,0.625000,0.918000,-5.810000,0.074050,0.177000,0.672500,0.105000,0.456000,104.008500,45:33,../data/albums/45:33.txt
1,A Moon Shaped Pool,Radiohead,"[alternative rock, art rock, melancholia, oxfo...",2016-05-08,XL Recordings,0.429545,0.406455,-11.596818,0.034264,0.673291,0.564455,0.158773,0.203018,125.244000,A_Moon_Shaped_Pool,../data/albums/A_Moon_Shaped_Pool.txt
2,Actor,St. Vincent,"[art pop, etherpop, indie rock, indietronica, ...",2009-05-05,4AD,0.484091,0.548455,-10.276273,0.039882,0.324571,0.433091,0.214909,0.329773,119.461091,Actor,../data/albums/Actor.txt
3,Al Mundo Azul,Mr. Twin Sister,[chillwave],2021-11-19,Twin Group,0.722100,0.698500,-6.106500,0.054780,0.275440,0.330606,0.191360,0.716500,125.701400,Al_Mundo_Azul,../data/albums/Al_Mundo_Azul.txt
4,Alligator,The National,"[chamber pop, indie folk, indie rock, indietro...",2005-04-11,Beggars Banquet,0.522154,0.740769,-6.816615,0.035585,0.284209,0.239670,0.125769,0.535831,125.053308,Alligator,../data/albums/Alligator.txt


### Add `song_regex` column

In [26]:
LIB['song_regex'] = '\[Trackname:\s[^\]]+\]'

In [27]:
LIB.head()

,album_title,artist,genres,release_date,label,mean_danceability,mean_energy,mean_loudness,mean_speechiness,mean_acousticness,mean_instrumentalness,mean_liveness,mean_valence,mean_tempo,album_name,source_file_path,song_regex
album_id,,,,,,,,,,,,,,,,,
0,45:33,LCD Soundsystem,"[alternative dance, alternative rock, art pop,...",2007-11-12,Parlophone UK,0.625000,0.918000,-5.810000,0.074050,0.177000,0.672500,0.105000,0.456000,104.008500,45:33,../data/albums/45:33.txt,\[Trackname:\s[^\]]+\]
1,A Moon Shaped Pool,Radiohead,"[alternative rock, art rock, melancholia, oxfo...",2016-05-08,XL Recordings,0.429545,0.406455,-11.596818,0.034264,0.673291,0.564455,0.158773,0.203018,125.244000,A_Moon_Shaped_Pool,../data/albums/A_Moon_Shaped_Pool.txt,\[Trackname:\s[^\]]+\]
2,Actor,St. Vincent,"[art pop, etherpop, indie rock, indietronica, ...",2009-05-05,4AD,0.484091,0.548455,-10.276273,0.039882,0.324571,0.433091,0.214909,0.329773,119.461091,Actor,../data/albums/Actor.txt,\[Trackname:\s[^\]]+\]
3,Al Mundo Azul,Mr. Twin Sister,[chillwave],2021-11-19,Twin Group,0.722100,0.698500,-6.106500,0.054780,0.275440,0.330606,0.191360,0.716500,125.701400,Al_Mundo_Azul,../data/albums/Al_Mundo_Azul.txt,\[Trackname:\s[^\]]+\]
4,Alligator,The National,"[chamber pop, indie folk, indie rock, indietro...",2005-04-11,Beggars Banquet,0.522154,0.740769,-6.816615,0.035585,0.284209,0.239670,0.125769,0.535831,125.053308,Alligator,../data/albums/Alligator.txt,\[Trackname:\s[^\]]+\]


### Reorder columns

In [28]:
LIB = LIB[[
    'album_name',
    'album_title',
    'artist',
    'source_file_path',
    'song_regex',
    'genres',
    'release_date',
    'label',
    'mean_danceability',
    'mean_energy',
    'mean_loudness',
    'mean_speechiness',
    'mean_acousticness',
    'mean_instrumentalness',
    'mean_liveness',
    'mean_valence',
    'mean_tempo'
]]

In [29]:
LIB.head()

,album_name,album_title,artist,source_file_path,song_regex,genres,release_date,label,mean_danceability,mean_energy,mean_loudness,mean_speechiness,mean_acousticness,mean_instrumentalness,mean_liveness,mean_valence,mean_tempo
album_id,,,,,,,,,,,,,,,,,
0,45:33,45:33,LCD Soundsystem,../data/albums/45:33.txt,\[Trackname:\s[^\]]+\],"[alternative dance, alternative rock, art pop,...",2007-11-12,Parlophone UK,0.625000,0.918000,-5.810000,0.074050,0.177000,0.672500,0.105000,0.456000,104.008500
1,A_Moon_Shaped_Pool,A Moon Shaped Pool,Radiohead,../data/albums/A_Moon_Shaped_Pool.txt,\[Trackname:\s[^\]]+\],"[alternative rock, art rock, melancholia, oxfo...",2016-05-08,XL Recordings,0.429545,0.406455,-11.596818,0.034264,0.673291,0.564455,0.158773,0.203018,125.244000
2,Actor,Actor,St. Vincent,../data/albums/Actor.txt,\[Trackname:\s[^\]]+\],"[art pop, etherpop, indie rock, indietronica, ...",2009-05-05,4AD,0.484091,0.548455,-10.276273,0.039882,0.324571,0.433091,0.214909,0.329773,119.461091
3,Al_Mundo_Azul,Al Mundo Azul,Mr. Twin Sister,../data/albums/Al_Mundo_Azul.txt,\[Trackname:\s[^\]]+\],[chillwave],2021-11-19,Twin Group,0.722100,0.698500,-6.106500,0.054780,0.275440,0.330606,0.191360,0.716500,125.701400
4,Alligator,Alligator,The National,../data/albums/Alligator.txt,\[Trackname:\s[^\]]+\],"[chamber pop, indie folk, indie rock, indietro...",2005-04-11,Beggars Banquet,0.522154,0.740769,-6.816615,0.035585,0.284209,0.239670,0.125769,0.535831,125.053308


## Build `CORPUS` table
Using the following OHCO:
[album_id, song_id, stanza_num, line_num, token_num]

This will require a bit of work given we store all of the lyrics at a song level in the .json

### Label song lyrics with song name
For simplicity we also go ahead and make these individual `.txt` files for each album

In [30]:
# # Loop over artists
# for artist in data.keys():
#     # Loop over albums
#     for album in data[artist]['Albums'].keys():
#         album_text = ''
#         album_name = re.sub(r' ', '_', album)
#         # Loop over songs
#         for track in data[artist]['Albums'][album]['tracklist'].keys():
#             lyrics = data[artist]['Albums'][album]['tracklist'][track]['lyrics']
#             lyrics = f'\n\n[Trackname: {track}]\n' + lyrics
#             album_text  = album_text + lyrics
#         with open(f'../data/albums/{album_name}.txt', 'w') as outfile:
#             print(f'Wrote album {album_name} to file {album_name}.txt')
#             print(f'{album_text}', file=outfile)
            

### Parse album `.txt` files into `CORPUS` table

### Define OHCO

In [31]:
OHCO = ['album_id', 'song_num', 'stanza_num', 'line_num', 'token_num']

### Find files

In [32]:
source_file_list = sorted(glob(f"../data/albums/*.*"))

### NLTK setup

In [33]:
nltk_resources = [
    'tokenizers/punkt', 
    'taggers/averaged_perceptron_tagger', 
    'corpora/stopwords', 
    'help/tagsets'
]

In [34]:
for rsc in nltk_resources:
    try:
        nltk.data.find(rsc)
    except IndexError:
        nltk.download(rsc)

### Function to tokenize `album.txt` files

In [35]:
def parse_album(album_id:int, album_name:str, song_regex:str):
    # Read in data and create LINES
    LINES = pd.DataFrame(open(f'../data/albums/{album_name}.txt', 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
    LINES.index.name = 'line_num'
    LINES.line_str = LINES.line_str.str.replace(r'\n', '', regex=True).str.strip()

    # Find song headers
    chap_pat = LIB.iloc[0].song_regex
    chap_lines = LINES.line_str.str.match(chap_pat, case=False)

    # Assign numbers to songs
    LINES.loc[chap_lines, 'song_num'] = [i for i in range(LINES.loc[chap_lines].shape[0])]

    # Forward fill song_num to following text lines
    # These are the lines for the song
    LINES.song_num = LINES.song_num.ffill()

    # Clean up
    LINES = LINES.dropna(subset=['song_num'])       # Remove everything before chapter 1
    LINES = LINES.loc[~chap_lines]                  # Remove chapter heading lines
    LINES.song_num = LINES.song_num.astype('int')   # Convert song_num to int

    # Start grouping into OHCO
    # Change songs into one big string
    SONGS = LINES.groupby(OHCO[1])\
        .line_str.apply(lambda x: '\n'.join(x))\
        .to_frame('song_str')
    SONGS['song_str'] = SONGS.song_str.str.strip()

    # Split songs into stanzas
    # This might not work for all lyrics, given the difference in formatting
    stanza_pat = r'\n\n+'
    STANZAS = SONGS['song_str'].str.split(stanza_pat, expand=True).stack()\
        .to_frame('stanza_str').sort_index()
    STANZAS.index.names = OHCO[1:3]

    # Clean up
    STANZAS['stanza_str'] = STANZAS['stanza_str'].str.strip()       # Strip leading and trailing spaces
    STANZAS = STANZAS[~STANZAS['stanza_str'].str.match(r'^\s*$')]   # Remove empty paragraphs

    # Split stanzas into lines
    line_pat = r'\n'
    LINES = STANZAS['stanza_str'].str.split(line_pat, expand=True).stack()\
        .to_frame('line_str')
    LINES.index.names = OHCO[1:4]

    # Clean up
    LINES = LINES[~LINES['line_str'].str.match(r'^\s*$')]   # Remove empty paragraphs
    LINES.line_str = LINES.line_str.str.strip()             # CRUCIAL TO REMOVE BLANK TOKENS

    # POS Tag based on lines
    # Using
    TOKENS = LINES.line_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
    TOKENS.index.names = OHCO[1:5]

    # Dropped in favor of NLTK processing above
    # # Split lines into tokens
    # token_pat = r"[\s',-]+"
    # TOKENS = LINES['line_str'].str.split(token_pat, expand=True).stack()\
    #     .to_frame('token_str')
    # TOKENS.index.names = OHCO[1:5]
    TOKENS = pd.concat({album_id: TOKENS}, names=[OHCO[0]])

    return TOKENS

### Loop over `LIB` and create `CORPUS`

In [36]:
albums = []

for album_id in LIB.index:
    albums.append(parse_album(album_id, LIB.iloc[album_id].album_name, LIB.iloc[album_id].song_regex))

CORPUS = pd.concat(albums).sort_index()

In [37]:
CORPUS

pos_tuple
album_id song_num stanza_num line_num token_num                
0        0        0          0        0             (Shame, NN)
                                      1                (on, IN)
                                      2              (you, PRP)
                             1        0             (Shame, NN)
                                      1                (on, IN)
...                                                         ...
91       9        0          40       3          (anywhere, RB)
                             41       0                (On, IN)
                                      1               (the, DT)
                                      2             (black, JJ)
                                      3            (screen, NN)

[199290 rows x 1 columns]

## Add `pos`, `token_str`, `term_str`, and `pos_group` to `CORPUS`

In [38]:
CORPUS['pos'] = CORPUS.pos_tuple.apply(lambda x: x[1])
CORPUS['token_str'] = CORPUS.pos_tuple.apply(lambda x: x[0])
CORPUS['term_str'] = CORPUS.token_str.str.lower().str.replace(r"\W+", "", regex=True)
CORPUS['pos_group'] = CORPUS.pos.str[:2]

In [39]:
CORPUS.sample(20)

,,,,,pos_tuple,pos,token_str,term_str,pos_group
album_id,song_num,stanza_num,line_num,token_num,,,,,
6,8,3,7,1,"(guitar, NN)",NN,guitar,guitar,NN
20,7,0,7,12,"(the, DT)",DT,the,the,DT
82,6,0,5,2,"(the, DT)",DT,the,the,DT
72,3,4,2,1,"(in, IN)",IN,in,in,IN
8,8,0,3,1,"(will, MD)",MD,will,will,MD
47,5,0,11,4,"(keeping, VBG)",VBG,keeping,keeping,VB
16,6,1,3,3,"(know, VBP)",VBP,know,know,VB
58,8,2,2,4,"(their, PRP$)",PRP$,their,their,PR
30,7,2,2,4,"(down, RP)",RP,down,down,RP


## Add song length to `LIB`

In [40]:
LIB['album_term_count'] = CORPUS.groupby('album_id').term_str.count()
LIB['album_character_count'] = CORPUS.groupby('album_id')['token_str']\
    .apply(lambda x: sum(x.str.len()))

## Drop any weirdness from `CORPUS`

In [41]:
CORPUS = CORPUS[CORPUS.term_str != '']

## Build `VOCAB` table

### Build core of the table

In [42]:
VOCAB = CORPUS.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()    
VOCAB['p'] = VOCAB.n/VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['max_pos'] = CORPUS[['term_str', 'pos']].value_counts()\
    .unstack(fill_value=0)\
    .idxmax(1)
VOCAB['max_pos_group'] = CORPUS[['term_str','pos_group']].value_counts()\
    .unstack(fill_value=0)\
    .idxmax(1)

### Add `sw`

In [43]:
sw = pd.DataFrame({'stop': 1}, index=nltk.corpus.stopwords.words('english'))
sw.index.name='term_str'

In [44]:
if 'stop' not in VOCAB.columns:
    VOCAB = VOCAB.join(sw)
    VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [45]:
VOCAB.head()

,n,n_chars,p,i,max_pos,max_pos_group,stop
term_str,,,,,,,
1,4,1,0.000020,15.604083,CD,CD,0
10cc,1,4,0.000005,17.604083,CD,CD,0
10th,5,4,0.000025,15.282155,CD,CD,0
1st,1,3,0.000005,17.604083,CD,CD,0
2,11,1,0.000055,14.144651,CD,CD,0


### Add `porter_stem`

In [46]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
VOCAB['porter_stem'] = VOCAB.apply(lambda x: stemmer.stem(x.name), 1)

## Generate `BOW`
We'll use song as the bag

In [47]:
BOW = get_BOW(CORPUS, 'song_num')

## Generate `DTM`, `dfidf`, and `TFIDF`
Went ahead and added `dfidf` to `VOCAB` here for simplicity

In [48]:
TFIDF, VOCAB['song_dfidf'], DTCM = get_TFIDF(BOW, 'max')

In [49]:
VOCAB.head()

,n,n_chars,p,i,max_pos,max_pos_group,stop,porter_stem,song_dfidf
term_str,,,,,,,,,
1,4,1,0.000020,15.604083,CD,CD,0,1,31.775920
10cc,1,4,0.000005,17.604083,CD,CD,0,10cc,9.943980
10th,5,4,0.000025,15.282155,CD,CD,0,10th,17.887960
1st,1,3,0.000005,17.604083,CD,CD,0,1st,9.943980
2,11,1,0.000055,14.144651,CD,CD,0,2,55.551839


In [50]:
DTCM.head()

term_str           1  10cc  10th  1st  2  23rd  2pac  3  30th  4  ...  à  \
album_id song_num                                                 ...      
0        0         0     0     0    0  0     0     0  0     0  0  ...  0   
         1         0     0     0    0  0     0     0  0     0  0  ...  0   
1        0         0     0     0    0  0     0     0  0     0  0  ...  0   
         1         0     0     0    0  0     0     0  0     0  0  ...  0   
         2         0     0     0    0  0     0     0  0     0  0  ...  0   

term_str           árbol  è  еh  еl  еnding  еxtrañando  еye  政権の腐敗  政権の腐敗敗  
album_id song_num                                                            
0        0             0  0   0   0       0           0    0      0       0  
         1             0  0   0   0       0           0    0      0       0  
1        0             0  0   0   0       0           0    0      0       0  
         1             0  0   0   0       0           0    0      0       0  
         2             0  0   0   0       0           0    0      0       0  

[5 rows x 9750 columns]

In [51]:
TFIDF.head()

term_str             1  10cc  10th  1st    2  23rd  2pac    3  30th    4  ...  \
album_id song_num                                                         ...   
0        0         0.0   0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
         1         0.0   0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
1        0         0.0   0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
         1         0.0   0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
         2         0.0   0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   

term_str             à  árbol    è   еh   еl  еnding  еxtrañando  еye  政権の腐敗  \
album_id song_num                                                              
0        0         0.0    0.0  0.0  0.0  0.0     0.0         0.0  0.0    0.0   
         1         0.0    0.0  0.0  0.0  0.0     0.0         0.0  0.0    0.0   
1        0         0.0    0.0  0.0  0.0  0.0     0.0         0.0  0.0    0.0   
         1         0.0    0.0  0.0  0.0  0.0     0.0         0.0  0.0    0.0   
         2         0.0    0.0  0.0  0.0  0.0     0.0         0.0  0.0    0.0   

term_str           政権の腐敗敗  
album_id song_num          
0        0            0.0  
         1            0.0  
1        0            0.0  
         1            0.0  
         2            0.0  

[5 rows x 9750 columns]

## Create `TFIDF_L2`
* Still using song as the bag
* Using top 1000 terms by `dfidf`
* Using POS list: `['NN', 'NNS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']`
* Taking terms with `n_chars > 3`
    * Looking for more significant words and there are some shorter words that don't seem that useful showing up in this list without this filter
    * It could be that nltk pos tagging struggled with line tagging as opposed to sentence tagging

### Generate `sig_terms`

In [52]:
# POS list
pos_list = ['NN', 'NNS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']
#pos_list = ['NN', 'NNS']

# Create significant term list
sig_terms = list(VOCAB.query(f'max_pos in {pos_list} and n_chars > 3')\
    .sort_values('song_dfidf', ascending=False)[:1000]\
    .index)

### Build and collapse `TFIDF_SUB` into `TFIDF_L2`

In [53]:
TFIDF_SUB = TFIDF[TFIDF.columns.intersection(sig_terms)]
TFIDF_SUB = TFIDF_SUB.groupby(OHCO[:2]).mean()
TFIDF_L2 = TFIDF_SUB.apply(lambda x: x / norm(x), 1)  # Pythagorean, AKA Euclidean

## Reorganize `SONG_LIB`

In [54]:
SONG_LIB = song_lib.merge(LIB['album_title'].reset_index(), left_on='album', right_on='album_title')
SONG_LIB = SONG_LIB.reset_index()
SONG_LIB = SONG_LIB.rename(columns={'index':'song_id'})
SONG_LIB = SONG_LIB.set_index(['album_id', 'song_id'])
SONG_LIB = SONG_LIB.sort_index(ascending=True)
SONG_LIB = SONG_LIB.reset_index('song_id', drop=True)
SONG_LIB = SONG_LIB.set_index(SONG_LIB.groupby(level=0).cumcount().rename('song_num'), append=True)

In [55]:
SONG_LIB.head()

title  \
album_id song_num                            
0        0                           45:33   
         1         Freak Out / Starry Eyes   
1        0                  Burn the Witch   
         1                     Daydreaming   
         2                      Decks Dark   

                                                           json_path  \
album_id song_num                                                      
0        0         data[LCD Soundsystem]['Albums'][45:33]['trackl...   
         1         data[LCD Soundsystem]['Albums'][45:33]['trackl...   
1        0         data[Radiohead]['Albums'][A Moon Shaped Pool][...   
         1         data[Radiohead]['Albums'][A Moon Shaped Pool][...   
         2         data[Radiohead]['Albums'][A Moon Shaped Pool][...   

                            artist               album  \
album_id song_num                                        
0        0         LCD Soundsystem               45:33   
         1         LCD Soundsystem               45:33   
1        0               Radiohead  A Moon Shaped Pool   
         1               Radiohead  A Moon Shaped Pool   
         2               Radiohead  A Moon Shaped Pool   

                                                              genres  \
album_id song_num                                                      
0        0         [alternative dance, alternative rock, art pop,...   
         1         [alternative dance, alternative rock, art pop,...   
1        0         [alternative rock, art rock, melancholia, oxfo...   
         1         [alternative rock, art rock, melancholia, oxfo...   
         2         [alternative rock, art rock, melancholia, oxfo...   

                  release_date          label  track_number  duration_ms  \
album_id song_num                                                          
0        0          2007-11-12  Parlophone UK             1      2764933   
         1          2007-11-12  Parlophone UK             2       742053   
1        0          2016-05-08  XL Recordings             1       220609   
         1          2016-05-08  XL Recordings             2       384438   
         2          2016-05-08  XL Recordings             3       281011   

                                                   audio_information  ...  \
album_id song_num                                                     ...   
0        0         {'danceability': 0.478, 'energy': 0.897, 'key'...  ...   
         1         {'danceability': 0.772, 'energy': 0.939, 'key'...  ...   
1        0         {'danceability': 0.54, 'energy': 0.847, 'key':...  ...   
         1         {'danceability': 0.214, 'energy': 0.263, 'key'...  ...   
         2         {'danceability': 0.557, 'energy': 0.501, 'key'...  ...   

                  loudness  mode  speechiness  acousticness  instrumentalness  \
album_id song_num                                                               
0        0          -6.554     1       0.0471         0.138             0.657   
         1          -5.066     0       0.1010         0.216             0.688   
1        0          -6.520     1       0.0297         0.303             0.272   
         1         -13.207     0       0.0336         0.968             0.853   
         2         -10.827     0       0.0269         0.666             0.837   

                   liveness  valence    tempo  time_signature  \
album_id song_num                                               
0        0            0.100    0.186  113.018               4   
         1            0.110    0.726   94.999               4   
1        0            0.109    0.620  148.936               4   
         1            0.126    0.113  137.561               3   
         2            0.117    0.271  139.149               4   

                          album_title  
album_id song_num                      
0        0                      45:33  
         1                      45:33  
1        0         A Moon Shaped Pool  


## Save tables to .csv

In [56]:
LIB.to_csv(f'../tables/LIB.csv', sep='|')
SONG_LIB.to_csv(f'../tables/SONG_LIB', sep='|')
CORPUS.to_csv(f'../tables/CORPUS.csv', sep='|')
VOCAB.to_csv(f'../tables/VOCAB.csv', sep='|')
BOW.to_csv(f'../tables/BOW_SONG.csv', sep='|')
DTCM.to_csv(f'../tables/DTCM_SONG.csv', sep='|')
TFIDF.to_csv(f'../tables/TFIDF_SONG.csv', sep='|')
TFIDF_L2.to_csv(f'../tables/TFIDF_L2_SONG.csv', sep='|')